In [1]:
import xmltodict, json
import requests
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv("ebay_api_key.env") #hide production key in environment file



APP_ID = os.getenv("EBAY_PROD_ID")


## URL 
HTTP call url to send the Production Client API. The Payload will modify the url below with a ? followed by an % and key value pairs. This is the most common and recommended way that ebay suggests users to send a request to the Finding API.

## HTTP Headers
Required headers to return a 200 response code. The top header is AuthnAuth security. As there is no need for a user token and the only interface is with my app, the only security required is for the app. The second row is the call be sent to the Finding API, findItemsAdvanced. ## THIS SHOULD BE A LINK DOG.   The third formats the respons as JSON.

## Payload
Modifies HTTP call. Search for knife in the Collectible-Modern-Factory-Manufactured-Folding-Knives that is of the Used Condition. This request returns 100 pages on page 1. 

In [2]:
url = "https://svcs.ebay.com/services/search/FindingService/v1" #url fir Finding API


headers = {
            "X-EBAY-SOA-SECURITY-APPNAME": APP_ID, #production client_id
            "X-EBAY-SOA-OPERATION-NAME": "findItemsAdvanced", #call_id
            "X-EBAY-API-RESPONSE-ENCODING": "JSON" #format respons as JSON
           }



payload = {
            "keywords":"knife", #knife in search bar
            "categoryId":"48818", #Collectible-Modern-Factory-Manufactured-Folding-Knives
            "itemFilter(0).name": "Condition",
            "itemFilter(0).value": "Used",
            "itemFilter(1).name": "ListingType",
            "itemFilter(1).value": "FixedPrice",
            "aspectFilter.aspectName(0)": "Brand",
            "aspectFilter.aspectValueName(0)": "Benchmade",
            "aspectFilter.aspectName(1)": "Brand",
            "aspectFilter.aspectValueName(1)": "Buck",
            "aspectFilter.aspectName(2)": "Brand",
            "aspectFilter.aspectValueName(2)": "Case",
            "aspectFilter.aspectName(3)": "Brand",
            "aspectFilter.aspectValueName(3)": "CRKT",
            "aspectFilter.aspectName(4)": "Brand",
            "aspectFilter.aspectValueName(4)": "Kershaw",
            "aspectFilter.aspectName(5)": "Brand",
            "aspectFilter.aspectValueName(5)": "Leatherman",
            "aspectFilter.aspectName(6)": "Brand",
            "aspectFilter.aspectValueName(6)": "Sog",
            "aspectFilter.aspectName(7)": "Brand",
            "aspectFilter.aspectValueName(7)": "Spyderco",
            "aspectFilter.aspectName(8)": "Brand",
            "aspectFilter.aspectValueName(8)": "Victorinox",
            "paginationInput.entriesPerPage":100, #pagination 
            "paginationInput.pageNumber":1
        
          }

In [4]:
r.url

'https://svcs.ebay.com/services/search/FindingService/v1?keywords=knife&categoryId=48818&itemFilter%280%29.name=Condition&itemFilter%280%29.value=Used&itemFilter%281%29.name=ListingType&itemFilter%281%29.value=FixedPrice&aspectFilter.aspectName%280%29=Brand&aspectFilter.aspectValueName%280%29=Benchmade&aspectFilter.aspectName%281%29=Brand&aspectFilter.aspectValueName%281%29=Buck&aspectFilter.aspectName%282%29=Brand&aspectFilter.aspectValueName%282%29=Case&aspectFilter.aspectName%283%29=Brand&aspectFilter.aspectValueName%283%29=CRKT&aspectFilter.aspectName%284%29=Brand&aspectFilter.aspectValueName%284%29=Kershaw&aspectFilter.aspectName%285%29=Brand&aspectFilter.aspectValueName%285%29=Leatherman&aspectFilter.aspectName%286%29=Brand&aspectFilter.aspectValueName%286%29=Sog&aspectFilter.aspectName%287%29=Brand&aspectFilter.aspectValueName%287%29=Spyderco&aspectFilter.aspectName%288%29=Brand&aspectFilter.aspectValueName%288%29=Victorinox&paginationInput.entriesPerPage=100&paginationInput.pag

In [3]:
r = requests.get(url, headers=headers, params=payload)
json_dict_100knivesp1 = xmltodict.parse(r.text)
knives_data_list_of_dicts = json_dict_100knivesp1['findItemsAdvancedResponse']['searchResult']['item']
total_pages = int(json_dict_100knivesp1['findItemsAdvancedResponse']['paginationOutput']['totalPages'])
total_pages

106

In [5]:
#create function for organizing API call
def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        
        keys = ['itemId', 'title', 'galleryURL', 
        'viewItemURL', 'postalCode', 'location', 
        'country', 'shippingInfo', 'sellingStatus', 
        'listingInfo', 'returnsAccepted', 'condition']
        
        for key in keys:
            prepared_data[key] = business_data.get(key, None)
    
       
        # Add to list if all values are present
        if all(prepared_data.values()):
            results.append(prepared_data)
    
    return results

In [6]:
#organize call with function
prepared_knives = prepare_data(knives_data_list_of_dicts)

In [7]:
total_pages

106

In [9]:
# Create an empty list for the full prepared dataset
full_dataset = []

for page in range(total_pages):
    # Add or update the "offset" key-value pair in url_params
    payload["paginationInput.pageNumber"] = page
    
    # Make the query and get the response
    response = requests.get(url, headers=headers, params=payload)
    
    # Get the response body in JSON format
    response_json = xmltodict.parse(r.text)
    
    # Get the list of businesses from the response_json
    knives_data_list_of_dicts = response_json['findItemsAdvancedResponse']['searchResult']['item']
    
    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(knives_data_list_of_dicts)
    
    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df = pd.DataFrame(full_dataset)

10494

In [10]:
df.to_csv('data/full_dataset.csv', index=False)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10494 entries, 0 to 10493
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   itemId           10494 non-null  object
 1   title            10494 non-null  object
 2   galleryURL       10494 non-null  object
 3   viewItemURL      10494 non-null  object
 4   postalCode       10494 non-null  object
 5   location         10494 non-null  object
 6   country          10494 non-null  object
 7   shippingInfo     10494 non-null  object
 8   sellingStatus    10494 non-null  object
 9   listingInfo      10494 non-null  object
 10  returnsAccepted  10494 non-null  object
 11  condition        10494 non-null  object
dtypes: object(12)
memory usage: 983.9+ KB


In [12]:
#Create row for converted Price of Knives in US dollars
price_list = []
for row in full_dataset:
    listed_price = np.float(row['sellingStatus']['convertedCurrentPrice']['#text'])
    price_list.append(listed_price)
    
df['price_in_US'] = price_list

In [13]:
import re

In [14]:
df['shipping_cost'] = df['shippingInfo'].apply(lambda x: re.findall("(\d+\S+\d)", json.dumps(x)))

In [15]:
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: ''.join(x))

In [16]:
df.drop(df[df['shipping_cost'] == ''].index, inplace=True)

In [17]:
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: np.float(x))

In [18]:
df['shipping_cost']

0        3.50
1        2.99
2        0.00
3        2.99
4        0.00
         ... 
10489    0.00
10490    0.00
10491    8.50
10492    0.00
10493    8.00
Name: shipping_cost, Length: 9646, dtype: float64

In [19]:
df['price_in_US']

0          7.25
1         22.99
2          8.53
3         16.99
4         36.00
          ...  
10489     14.00
10490     32.00
10491     30.00
10492     20.99
10493    125.00
Name: price_in_US, Length: 9646, dtype: float64

In [20]:
df['converted_price'] = df['shipping_cost'] + df['price_in_US']

In [21]:
df['converted_price']

0         10.75
1         25.98
2          8.53
3         19.98
4         36.00
          ...  
10489     14.00
10490     32.00
10491     38.50
10492     20.99
10493    133.00
Name: converted_price, Length: 9646, dtype: float64

In [22]:
df.sample(20)

,itemId,title,galleryURL,viewItemURL,postalCode,location,country,shippingInfo,sellingStatus,listingInfo,returnsAccepted,condition,price_in_US,shipping_cost,converted_price
2899,304554396037,Victorinox original Minichamp Swiss Army knife...,https://i.ebayimg.com/thumbs/images/g/tE0AAOSw...,https://www.ebay.com/itm/Victorinox-original-M...,857**,"Tucson,AZ,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",17.00,4.00,21.00
7325,234613567439,Lightly Used Benchmade 535 Bugout Folding Knife,https://i.ebayimg.com/thumbs/images/g/6b8AAOSw...,https://www.ebay.com/itm/Lightly-Used-Benchmad...,360**,"Prattville,AL,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",false,"{'conditionId': '3000', 'conditionDisplayName'...",125.00,8.00,133.00
3110,165560377525,victorinox swiss army knife officier suisse 13...,https://i.ebayimg.com/thumbs/images/g/3ucAAOSw...,https://www.ebay.com/itm/victorinox-swiss-army...,956**,"Pine Grove,CA,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",34.00,0.00,34.00
2230,144636534294,Spyderco Tenacious Pocket Knife,https://i.ebayimg.com/thumbs/images/g/6HMAAOSw...,https://www.ebay.com/itm/Spyderco-Tenacious-Po...,462**,"Indianapolis,IN,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",true,"{'conditionId': '3000', 'conditionDisplayName'...",34.00,0.00,34.00
10092,384966450296,Benchmade 417BK Folding Knife,https://i.ebayimg.com/thumbs/images/g/5icAAOSw...,https://www.ebay.com/itm/Benchmade-417BK-Foldi...,836**,"Nampa,ID,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",true,"{'conditionId': '3000', 'conditionDisplayName'...",170.00,15.00,185.00
8133,255609660171,KERSHAW POCKET KNIFE 1329 ASSISTED OPENING,https://i.ebayimg.com/thumbs/images/g/ztkAAOSw...,https://www.ebay.com/itm/KERSHAW-POCKET-KNIFE-...,242**,"Abingdon,VA,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'true', 'buyItNowAvailabl...",true,"{'conditionId': '3000', 'conditionDisplayName'...",17.94,4.99,22.93
53,203912881164,"Victorinox Huntsman Swiss Army Knife, Saw Scis...",https://i.ebayimg.com/thumbs/images/g/bDcAAOSw...,https://www.ebay.com/itm/Victorinox-Huntsman-S...,492**,"Jackson,MI,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",18.95,0.00,18.95
3501,402908964795,"Vtg Kershaw Kai 5100 Seki Japan 4"" Gentleman F...",https://i.ebayimg.com/thumbs/images/g/o78AAOSw...,https://www.ebay.com/itm/Vtg-Kershaw-Kai-5100-...,838**,"Post Falls,ID,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",24.95,0.00,24.95
10376,334467087808,Victorinox Classic SD Swiss Army Knife 3 Tool ...,https://i.ebayimg.com/thumbs/images/g/3f4AAOSw...,https://www.ebay.com/itm/Victorinox-Classic-SD...,303**,"Atlanta,GA,USA",US,"{'shippingServiceCost': {'@currencyId': 'USD',...","{'currentPrice': {'@currencyId': 'USD', '#text...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,"{'conditionId': '3000', 'conditionDisplayName'...",8.59,0.00,8.59
9304,204005131701,Kershaw Assisted Ope

In [23]:
df.describe()

,price_in_US,shipping_cost,converted_price
count,9646.000000,9646.000000,9646.000000
mean,44.699670,1.736813,46.436484
std,68.160162,3.046098,68.771557
min,2.800000,0.000000,6.600000
25%,17.000000,0.000000,18.000000
50%,26.950000,0.000000,27.990000
75%,45.000000,3.500000,47.990000
max,579.000000,15.000000,579.000000


In [24]:
df.to_csv('data/full_dataset.csv', index=False)

In [ ]:
# # Make a new list to hold results
#     results = []
    
#     for business_data in full_dataset:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
        
#         keys = ['itemId', 'title', 'galleryURL', 
#         'viewItemURL', 'postalCode', 'location', 
#         'country', 'shippingInfo', 'sellingStatus', 
#         'listingInfo', 'returnsAccepted', 'condition']
        
#         for key in keys:
#             prepared_data[key] = business_data.get(key, None)
    
       
#         # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
#     return results

In [ ]:
# #create function for organizing API call
# def prepare_data(data_list):
#     """
#     This function takes in a list of dictionaries and prepares it
#     for analysis
#     """
    
#     # Make a new list to hold results
#     results = []
    
#     for business_data in data_list:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
        
#         keys = ['shippingServiceCost]
        
#         for key in keys:
#             prepared_data[key] = business_data.get(key, None)
    
       
#         # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
    return results